In [1]:
def ganancia(model, X, y, prop):
    y_pred = model.predict(X)
    return f1_score(y, y_pred)

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [ ]:
# Creador clase ternaria

# df_parallel = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

# start_time = time.time()

# # Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

# df_ternaria= duckdb.query('''

#     WITH siguiente_mes AS (

#         SELECT

#             numero_de_cliente,

#             foto_mes,

#             -- Obtener los meses siguientes en los que el cliente está presente

#             LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

#             LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

#         FROM df_parallel

#     )

#     SELECT

#         numero_de_cliente,

#         foto_mes,

#         CASE

#             -- Si el cliente está en los dos meses consecutivos siguientes

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

#             -- Si el cliente no está en el siguiente mes

#             WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

#             -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

#             ELSE NULL

#         END AS clase_ternaria

#     FROM siguiente_mes

#     ORDER BY numero_de_cliente, foto_mes

#     ''')

# df_ternaria = df_ternaria.to_df()
# df_ternaria = df_ternaria[['numero_de_cliente', 'foto_mes', 'clase_ternaria']]


# df_parallel = df_parallel.merge(df_ternaria, on=['numero_de_cliente', 'foto_mes'], how='left')

# df_train = df_parallel


In [ ]:

# con = duckdb.connect(database=':memory:', read_only=False)

# ruta_csv = r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_2\competencia_02_f_e_lags.parquet"

# con.execute(f"CREATE TABLE df_train AS SELECT * FROM read_parquet('{ruta_csv}')")

In [2]:
dicc_psi = {'dataset_clase_ternaria':['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'mcajeros_propios_descuentos',
 'mpayroll2',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad']
            ,
  'dataset_clase_ternaria_para_transformar':['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad'],
  
  
  'dataset_clase_ternaria_para_sacar': ['mcajeros_propios_descuentos', 'mpayroll2'],
  
'dataset_lags_deltas_y_clase_ternaria':['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Visa_Finiciomora_delta_2',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'mcajeros_propios_descuentos',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll2_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx',
 'Visa_fultimo_cierre_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Master_fultimo_cierre_delta_2',
 'Visa_fultimo_cierre_delta_2',
 'internet',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_delta_1',
 'ccomisiones_otras_delta_2',
 'mcomisiones_mantenimiento',
 'mrentabilidad_annual_lag_2',
 'Visa_Finiciomora_lag_1',
 'mcomisiones_mantenimiento_lag_2',
 'ccomisiones_otras_delta_1',
 'mrentabilidad_annual_lag_1',
 'mcomisiones_otras_lag_2',
 'mrentabilidad_lag_2',
 'Master_Fvencimiento_lag_1',
 'mcomisiones_otras_lag_1',
 'cforex_lag_2',
 'mforex_sell_lag_2',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'Visa_mconsumosdolares_lag_2',
 'mrentabilidad_annual_delta_2',
 'cforex_sell_lag_2',
 'mcomisiones_otras',
 'mcomisiones',
 'mforex_sell_lag_1',
 'mrentabilidad_lag_1',
 'cforex_lag_1',
 'Visa_mconsumosdolares_lag_1',
 'mrentabilidad_annual_delta_1',
 'cpayroll_trx_delta_1',
 'chomebanking_transacciones_lag_2',
 'mtransferencias_recibidas_delta_1',
 'mtransferencias_recibidas_lag_2',
 'mautoservicio_lag_2',
 'Visa_mfinanciacion_limite_delta_2',
 'cforex_sell_lag_1',
 'ctransferencias_recibidas_lag_2',
 'active_quarter_delta_2',
 'mtransferencias_recibidas',
 'cforex_delta_2',
 'Visa_mlimitecompra_lag_2',
 'ccomisiones_otras_lag_2',
 'Visa_mlimitecompra_delta_2',
 'mforex_sell_delta_2',
 'Visa_mfinanciacion_limite_delta_1',
 'mtransferencias_recibidas_delta_2',
 'cforex_sell_delta_2',
 'mtarjeta_visa_consumo_lag_2',
 'mrentabilidad',
 'mtransferencias_emitidas_lag_2',
 'Visa_mfinanciacion_limite_lag_2',
 'chomebanking_transacciones_lag_1',
 'mautoservicio_lag_1'],
'delta_lags_y_clase_ternaria_para_transformar':['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx',
 'Visa_fultimo_cierre_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Master_fultimo_cierre_delta_2',
 'Visa_fultimo_cierre_delta_2',
 'internet',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_delta_1',
 'ccomisiones_otras_delta_2',
 'mcomisiones_mantenimiento',
 'mrentabilidad_annual_lag_2',
 'Visa_Finiciomora_lag_1',
 'mcomisiones_mantenimiento_lag_2',
 'ccomisiones_otras_delta_1',
 'mrentabilidad_annual_lag_1',
 'mcomisiones_otras_lag_2',
 'mrentabilidad_lag_2',
 'Master_Fvencimiento_lag_1',
 'mcomisiones_otras_lag_1',
 'mforex_sell_lag_2',
 'mrentabilidad_annual',
 'mrentabilidad_annual_delta_2',
 'cforex_sell_lag_2',
 'mcomisiones_otras',
 'mcomisiones',
 'mrentabilidad_lag_1',
 'mrentabilidad_annual_delta_1',
 'chomebanking_transacciones_lag_2',
 'mtransferencias_recibidas_lag_2',
 'mautoservicio_lag_2',
 'Visa_mfinanciacion_limite_delta_2',
 'active_quarter_delta_2',
 'mtransferencias_recibidas',
 'Visa_mlimitecompra_lag_2',
 'ccomisiones_otras_lag_2',
 'Visa_mfinanciacion_limite_delta_1',
 'mtarjeta_visa_consumo_lag_2',
 'mrentabilidad',
 'mtransferencias_emitidas_lag_2',
 'Visa_mfinanciacion_limite_lag_2',
 'chomebanking_transacciones_lag_1',
 'mautoservicio_lag_1'],

'delta_lags_y_clase_ternaria_para_sacar': ['Visa_Finiciomora_delta_2',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mcajeros_propios_descuentos',
 'mpayroll2_lag_1',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'cforex_lag_2',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento',
 'Visa_mconsumosdolares_lag_2',
 'mforex_sell_lag_1',
 'cforex_lag_1',
 'Visa_mconsumosdolares_lag_1',
 'cpayroll_trx_delta_1',
 'mtransferencias_recibidas_delta_1',
 'cforex_sell_lag_1',
 'ctransferencias_recibidas_lag_2',
 'cforex_delta_2',
 'Visa_mlimitecompra_delta_2',
 'mforex_sell_delta_2',
 'mtransferencias_recibidas_delta_2',
 'cforex_sell_delta_2'], 

 'lista_light_gbm_feature_importance':['ccajas_depositos_lag_1',
 'Master_msaldopesos_delta_2',
 'Master_msaldopesos_delta_1',
 'Master_msaldototal_delta_2',
 'Master_msaldodolares_delta_2',
 'Master_mpagado_delta_1',
 'Master_mpagospesos_delta_1',
 'Master_mpagospesos_delta_2',
 'mprestamos_hipotecarios_lag_2',
 'Master_mpagado_delta_2',
 'Master_madelantodolares_delta_1',
 'Master_madelantodolares_delta_2',
 'Visa_madelantodolares_delta_1',
 'Visa_madelantodolares_delta_2',
 'Master_mconsumototal_delta_2',
 'Master_cconsumos_delta_1',
 'Visa_mconsumototal_delta_1',
 'Visa_mconsumototal_delta_2',
 'Master_mpagado_lag_1',
 'Master_madelantodolares_lag_2',
 'Master_madelantodolares_lag_1',
 'Master_madelantopesos_lag_2',
 'Master_msaldodolares',
 'catm_trx_other',
 'ccajas_extracciones',
 'ccajas_depositos',
 'Master_madelantopesos_lag_1',
 'Master_mconsumototal_lag_2',
 'Master_mconsumototal_lag_1',
 'Master_mpagosdolares_lag_2',
 'Master_cadelantosefectivo_lag_1',
 'Master_mpagado_lag_2',
 'Master_mpagospesos_lag_2',
 'Master_mpagospesos_lag_1',
 'Master_madelantopesos',
 'Master_mpagado',
 'Master_cadelantosefectivo',
 'Master_mconsumototal',
 'cprestamos_hipotecarios_lag_2',
 'Visa_madelantodolares_lag_1',
 'mpagodeservicios_lag_2',
 'cprestamos_hipotecarios_lag_1',
 'mpagomiscuentas_lag_1',
 'cpagomiscuentas_lag_1',
 'ccajeros_propios_descuentos_lag_1',
 'cpagomiscuentas_lag_2',
 'mtarjeta_master_consumo_lag_1',
 'Master_cadelantosefectivo_lag_2',
 'Master_mlimitecompra',
 'Master_mpagosdolares_lag_1',
 'ctarjeta_master_descuentos_lag_1',
 'ctarjeta_visa_descuentos_lag_2',
 'ctarjeta_visa_descuentos_lag_1',
 'ccajeros_propios_descuentos_lag_2',
 'ccomisiones_mantenimiento_lag_1',
 'ccajas_depositos_lag_2',
 'ccajas_extracciones_lag_1',
 'Master_msaldodolares_delta_1',
 'ccajas_extracciones_lag_2',
 'ccajas_otras_lag_1',
 'ccajas_otras_lag_2',
 'matm_lag_1',
 'catm_trx_other_lag_1',
 'catm_trx_other_lag_2',
 'matm_other_lag_1',
 'ccajas_transacciones_lag_2',
 'Master_msaldopesos_lag_1',
 'Master_msaldodolares_lag_1',
 'Master_mconsumospesos_lag_1',
 'Master_msaldodolares_lag_2',
 'Master_mconsumospesos_lag_2',
 'Master_mconsumosdolares_lag_1',
 'Master_mconsumosdolares_lag_2',
 'cplazo_fijo_lag_1',
 'mplazo_fijo_dolares_lag_1',
 'mplazo_fijo_pesos_lag_1',
 'matm_delta_2',
 'catm_trx_other_delta_1',
 'catm_trx_other_delta_2',
 'matm_other_delta_2',
 'Master_madelantopesos_delta_1',
 'Master_madelantopesos_delta_2',
 'ccheques_depositados_rechazados_lag_2',
 'ccheques_depositados_rechazados_lag_1',
 'mcheques_emitidos_lag_1',
 'ccheques_emitidos_lag_1',
 'mcheques_depositados_lag_1',
 'mcheques_depositados_lag_2',
 'ccheques_depositados_lag_2',
 'mforex_sell',
 'mtarjeta_master_descuentos',
 'cforex_sell',
 'cforex',
 'cforex_buy',
 'ctarjeta_master_descuentos',
 'ccajeros_propios_descuentos',
 'ctarjeta_visa_descuentos',
 'mtarjeta_visa_descuentos',
 'ccheques_depositados_rechazados',
 'mcheques_depositados',
 'ccheques_depositados',
 'Visa_cadelantosefectivo_lag_1',
 'Visa_mconsumototal_lag_2',
 'Visa_mconsumototal_lag_1',
 'Visa_madelantodolares_lag_2',
 'mcheques_depositados_rechazados',
 'mtarjeta_master_descuentos_lag_1',
 'mtarjeta_master_descuentos_lag_2',
 'ctarjeta_master_descuentos_lag_2',
 'cpayroll_trx_delta_2',
 'mprestamos_prendarios_delta_2',
 'mprestamos_hipotecarios_delta_2',
 'mprestamos_hipotecarios_delta_1',
 'cprestamos_hipotecarios_delta_2',
 'cpayroll2_trx_lag_1',
 'cpayroll2_trx_lag_2',
 'mpagodeservicios_delta_2',
 'mplazo_fijo_pesos_lag_2',
 'mpagodeservicios_delta_1',
 'cpayroll2_trx_delta_1',
 'ctarjeta_master_debitos_automaticos_delta_1',
 'ctarjeta_master_debitos_automaticos_lag_1',
 'mpagomiscuentas_delta_2',
 'mpagodeservicios_lag_1',
 'cpagomiscuentas_delta_2',
 'ccajeros_propios_descuentos_delta_2',
 'mttarjeta_master_debitos_automaticos_lag_2',
 'cpagodeservicios_lag_1',
 'mttarjeta_master_debitos_automaticos_lag_1',
 'ctarjeta_master_debitos_automaticos_lag_2',
 'mtarjeta_visa_descuentos_delta_1',
 'ctarjeta_visa_descuentos_delta_2',
 'ctarjeta_visa_descuentos_delta_1',
 'mcajeros_propios_descuentos_delta_2',
 'cforex_delta_1',
 'cpayroll2_trx_delta_2',
 'mpayroll2_delta_1',
 'mpayroll2_delta_2',
 'mttarjeta_master_debitos_automaticos_delta_2',
 'ctarjeta_visa_transacciones_delta_2',
 'cprestamos_hipotecarios_delta_1',
 'ccajas_transacciones_lag_1',
 'mcheques_emitidos_rechazados_lag_2',
 'ccallcenter_transacciones_lag_2',
 'mcheques_depositados_rechazados_lag_2',
 'mcheques_depositados_rechazados_lag_1',
 'cseguro_accidentes_personales',
 'cseguro_vivienda',
 'cseguro_auto',
 'mextraccion_autoservicio_delta_2',
 'cextraccion_autoservicio_delta_1',
 'cforex_buy_delta_2',
 'mforex_buy_delta_1',
 'mforex_buy_delta_2',
 'mtarjeta_master_descuentos_delta_2',
 'mtarjeta_master_descuentos_delta_1',
 'ctarjeta_master_descuentos_delta_2',
 'ccheques_emitidos_delta_2',
 'ccheques_depositados_delta_2',
 'mcheques_depositados_delta_2',
 'ccheques_depositados_rechazados_delta_1',
 'cpayroll2_trx',
 'ccajas_transacciones_delta_2',
 'ccheques_emitidos_rechazados_delta_2',
 'mcheques_depositados_rechazados_delta_1',
 'ccheques_emitidos_rechazados_delta_1',
 'ccheques_depositados_rechazados_delta_2',
 'ccajas_extracciones_delta_2',
 'Visa_madelantodolares',
 'Visa_cadelantosefectivo_delta_2',
 'Visa_mconsumototal',
 'Visa_cadelantosefectivo',
 'active_quarter_lag_1',
 'Master_mpagosdolares_delta_2',
 'Master_mconsumototal_delta_1',
 'mcheques_depositados_delta_1',
 'ctarjeta_visa_debitos_automaticos',
 'mttarjeta_visa_debitos_automaticos',
 'Visa_msaldodolares_delta_2',
 'Visa_msaldodolares_delta_1',
 'Visa_msaldopesos_delta_1',
 'cforex_buy_lag_1',
 'mtransferencias_recibidas_lag_1',
 'cextraccion_autoservicio_lag_1',
 'mforex_buy_lag_1',
 'cforex_buy_lag_2',
 'mextraccion_autoservicio_lag_2',
 'active_quarter_lag_2',
 'ctarjeta_visa_transacciones_lag_2',
 'ccajas_otras_delta_2',
 'matm_delta_1',
 'Master_cconsumos_delta_2',
 'cextraccion_autoservicio_lag_2',
 'ctransferencias_recibidas_lag_1',
 'cprestamos_hipotecarios',
 'ctarjeta_master_transacciones',
 'mplazo_fijo_pesos',
 'cinversion1',
 'Master_cadelantosefectivo_delta_2',
 'Visa_msaldototal_delta_1',
 'cinversion2',
 'Master_cadelantosefectivo_delta_1',
 'Master_mconsumosdolares_delta_1',
 'Visa_madelantopesos_delta_2',
 'ccuenta_corriente',
 'Master_mlimitecompra_delta_1',
 'Visa_mconsumospesos_delta_2',
 'Visa_mlimitecompra_delta_1',
 'Visa_madelantopesos_delta_1',
 'cliente_vip']
}